# Create pre-train and finetuning dataset text files

In [3]:
!python preprocessing_python/text_generator.py \
        --file_path base-ehr-data/base_red3.csv \
        --output_folder /data \
        --create_pretrain_text_file

Producing text file from csv dataset: 100%|██| 680/680 [00:01<00:00, 563.09it/s]


In [4]:
!python preprocessing_python/text_generator.py \
        --file_path /kaggle/input/base-ehr-data/base_red3.csv \
        --output_folder /data \
        --output_name finetuning_dataset.txt \
        --create_finetuning_text_data

Reading input file: 100%|████████████████████| 680/680 [00:01<00:00, 440.03it/s]
Creating dataset for finetuning: 11481it [00:00, 150657.02it/s]


# Both mlm_nsp train and eval

### Only our pre-train

In [5]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm_nsp \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/our_pretrain_only_mlm_nsp \
      --use_pretrained_bert

2024-02-25 17:08:57.050125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 17:08:57.050256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 17:08:57.165754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm_nsp
tokenizer_config.json: 100%|██████████████████| 48.0/48.0 [00:00<00:00, 209kB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 1.40MB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 1.87MB/s]
config.json: 100%|█████████████████

In [6]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/our_pretrain_only_mlm_nsp \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/our_pretrain_only_mlm_nsp

2024-02-25 18:29:30.040896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:29:30.040952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:29:30.042445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_mlm_nsp/pre_trained_model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

### Bert pretrained without further pretrain

In [7]:
!python run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks mlm_nsp \
      --input_file /data/dataset_text.txt \
      --use_pretrained_bert

2024-02-25 18:38:52.780445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:38:52.780501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:38:52.781933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating:   0%|                                       | 0/217 [00:00<?, ?it/s]/kaggle/working/bert_medical_records/load_dataset.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val

In [8]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/just_bert_mlm_nsp \
      --use_pretrained_bert

2024-02-25 18:42:12.755189: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:42:12.755243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:42:12.756695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_mlm_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

### Bert pretrained with our pretrain

In [9]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/bert_mlm_nsp_with_pretrain \
      --use_pretrained_bert

2024-02-25 18:51:36.117446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:51:36.117508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:51:36.118994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_mlm_nsp_with_pretrain
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaske

In [10]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/bert_mlm_nsp_with_pretrain \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/bert_mlm_nsp_with_pretrain

2024-02-25 18:56:52.571502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 18:56:52.571562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 18:56:52.573135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_mlm_nsp_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_mlm_nsp_with_pretrain/pre_trained_model and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dens

# Only mlm pretrain

### Only our pre-train

In [11]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/our_pretrain_only_mlm

2024-02-25 19:06:20.334776: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:06:20.334834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:06:20.336348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 854      /      854
[00:00:00] Count pairs                    ██████████████████ 854      /      854
[00:00:00] Compute merges                 ██████████████████ 1248     /     1248
Saving the tokenize

In [12]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/our_pretrain_only_mlm \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/our_pretrain_only_mlm

2024-02-25 19:11:37.187614: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:11:37.187672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:11:37.189258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_mlm/pre_trained_model and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
Y

### Bert pretrained without further pretrain

In [13]:
!python run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --input_file /data/dataset_text.txt \
      --use_pretrained_bert

2024-02-25 19:21:02.799267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:21:02.799317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:21:02.800865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture 

In [14]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/just_bert_mlm_only \
      --use_pretrained_bert

2024-02-25 19:21:29.921874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:21:29.921932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:21:29.923391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_mlm_only
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

### Bert pretrained with our pretrain

In [15]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/bert_only_mlm_with_pretrain \
      --use_pretrained_bert

2024-02-25 19:30:55.807048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:30:55.807115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:30:55.808624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_only_mlm_with_pretrain
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMask

In [16]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/bert_only_mlm_with_pretrain \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/bert_only_mlm_with_pretrain

2024-02-25 19:36:13.637269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:36:13.637326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:36:13.638810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_only_mlm_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_only_mlm_with_pretrain/pre_trained_model and are newly initialized: ['classifier.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifie

# Only NSP

### Only our pretrain

In [17]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/our_pretrain_only_nsp

2024-02-25 19:45:44.154815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 19:45:44.154872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 19:45:44.156380: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_nsp
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 854      /      854
[00:00:00] Count pairs                    ██████████████████ 854      /      854
[00:00:00] Compute merges                 ██████████████████ 1248     /     1248
Saving the tokenize

In [18]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/our_pretrain_only_nsp \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/our_pretrain_only_nsp

2024-02-25 20:49:11.520927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 20:49:11.520985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 20:49:11.522490: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_nsp/pre_trained_model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream t

### Bert pretrained with no further pretrain

In [19]:
!python run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --input_file /data/dataset_text.txt \
      --use_pretrained_bert

2024-02-25 20:58:42.447573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 20:58:42.447647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 20:58:42.449060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating:   0%|                                       | 0/217 [00:00<?, ?it/s]/kaggle/working/bert_medical_records/load_dataset.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val

In [20]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/just_bert_only_nsp \
      --use_pretrained_bert

2024-02-25 20:59:46.772285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 20:59:46.772343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 20:59:46.773732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_only_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

### Bert pretrained with our pretrain

In [21]:
!python run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --num_epochs 8 \
      --input_file /data/dataset_text.txt \
      --output_dir /output/bert_nsp_with_pretrain

2024-02-25 21:09:19.101021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 21:09:19.101083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 21:09:19.102560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_nsp_with_pretrain
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 854      /      854
[00:00:00] Count pairs                    ██████████████████ 854      /      854
[00:00:00] Compute merges                 ██████████████████ 1248     /     1248
Saving the tokeniz

In [22]:
!python run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /output/bert_nsp_with_pretrain \
      --input_file /data/finetuning_dataset.txt \
      --output_dir /output/bert_nsp_with_pretrain

2024-02-25 22:12:49.279823: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 22:12:49.279881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 22:12:49.281396: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_nsp_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_nsp_with_pretrain/pre_trained_model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream